In [83]:
from web3 import Web3, HTTPProvider
from eth_abi import encode
from eth_utils import keccak
import json

In [84]:
OWNER_ADDRESS = "0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266"
OWNER_PRIVATE_KEY = "0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80"

In [85]:
SOLUTION_ADDRESSES = {
  "AdditionSolution": (1, "0x8464135c8F25Da09e49BC8782676a84730C318bC"),
  "MaxValueSolution": (2, "0x71C95911E9a5D330f4D621842EC243EE1343292e"),
}

PROBLEM_ID, SOLUTION_ADDRESS = SOLUTION_ADDRESSES["AdditionSolution"]

SOLVER_ADDRESS = "0x70997970C51812dc3A010C7d01b50e0d17dc79C8"
SOLVER_PRIVATE_KEY = "0x59c6995e998f97a5a0044966f0945389dc9e86dae88c7a8412f4603b6b78690d"

In [86]:
CP_ABI_PATH = "../out/ProgrammingCompetition.sol/ProgrammingCompetition.json"

w3 = Web3(HTTPProvider("http://localhost:8545"))

with open(CP_ABI_PATH, 'r') as f:
    contract_abi = json.load(f)['abi']

contract = w3.eth.contract(address="0x5FbDB2315678afecb367f032d93F642f64180aa3", abi=contract_abi)

In [87]:
def create_expected_result_hash():
    encoded_2 = encode(['uint256'], [2])
    encoded_4 = encode(['uint256'], [4])
    encoded_12 = encode(['uint256'], [12])

    all_results = encoded_2 + encoded_4 + encoded_12

    expected_results_hash = keccak(all_results).hex()

    print(f"Expected results hash: {expected_results_hash}")

In [88]:
def register_problem():
    expected_results_hash = "0x1f7d96dc808e221cdcfee683a0ce9dfc707f5a1ee10ec6febbcb3621cb1ac856"
    title = "Addition Problem"
    description = "Solve the addition of two numbers. The expected results are the sum of the inputs provided in the test cases."

    test_case_1 = encode(['uint256', 'uint256'], [1, 1])
    test_case_2 = encode(['uint256', 'uint256'], [2, 2])
    test_case_3 = encode(['uint256', 'uint256'], [5, 7])

    test_cases = [
        test_case_1,
        # "0x00000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000001",
        test_case_2,
        test_case_3
    ]

    # Build transaction WITHOUT 'gas'
    tx = contract.functions.registerProblem(
        PROBLEM_ID,
        expected_results_hash,
        test_cases,
        title,
        description
    ).build_transaction({
        'from': OWNER_ADDRESS,
        'value': w3.to_wei(0.1, 'ether'),
        'nonce': w3.eth.get_transaction_count(OWNER_ADDRESS),
    })

    try:
        gas_estimate = w3.eth.estimate_gas({
            'from': tx['from'],
            'to': tx['to'],
            'data': tx['data'],
            'value': tx['value'],
        })
        print(f"Estimated gas: {gas_estimate}")
        tx['gas'] = gas_estimate
    except Exception as e:
        message = str(e)
        if "revert reason:" in message:
            reason = message.split("revert reason:")[1].strip()
            print(f"Revert reason: {reason}")
        else:
            print(f"Transaction would revert. Error: {message}")

    signed_tx = w3.eth.account.sign_transaction(tx, private_key=OWNER_PRIVATE_KEY)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    print(f"Transaction sent: {tx_hash.hex()}")

    receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    if receipt.status == 1:
        print("Transaction was successful!")
    else:
        print("Transaction failed!")


# register_problem()

In [89]:
problem_id = PROBLEM_ID
problem_exists = contract.functions.problemExists(problem_id).call()
if problem_exists:
    print(f"Problem {problem_id} exists.")

    prize = contract.functions.getProblemPrize(problem_id).call()
    print(f"Prize for problem {problem_id}: {Web3.from_wei(prize, 'ether')} ETH")
    
    is_solved = contract.functions.isProblemSolved(problem_id).call()
    print(f"Is problem {problem_id} solved? {'Yes' if is_solved else 'No'}")

Problem 1 exists.
Prize for problem 1: 100000000 ETH
Is problem 1 solved? No


In [90]:
SENDER_ADDRESS = "0xa0Ee7A142d267C1f36714E4a8F75612F20a79720"
SENDER_PRIVATE_KEY = "0x2a871d0798f97d79848a013d4936a73bf4cc922c825d33c1cf7073dff6d409c6"

solution_balance = w3.eth.get_balance(SOLUTION_ADDRESS)
print(f"Solution contract balance: {w3.from_wei(solution_balance, 'ether')} ETH")

tx = {
    'from': SENDER_ADDRESS,
    'to': SOLUTION_ADDRESS,
    'value': w3.to_wei(10, 'ether'),
    'gasPrice': w3.eth.gas_price,
    'nonce': w3.eth.get_transaction_count(SENDER_ADDRESS),
}
tx['gas'] = w3.eth.estimate_gas(tx)
signed_tx = w3.eth.account.sign_transaction(tx, private_key=SENDER_PRIVATE_KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
print(f"Sent 10 ETH to solution contract. Tx hash: {tx_hash.hex()}")

receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print(f"Transaction receipt status: {receipt.status}")

solution_balance_after = w3.eth.get_balance(SOLUTION_ADDRESS)
print(f"Solution contract balance after: {w3.from_wei(solution_balance_after, 'ether')} ETH")

Solution contract balance: 0 ETH
Sent 10 ETH to solution contract. Tx hash: fb09d465a41763bfad7204a88c8b830b4853634e14c1c236567c4222b59465bc
Transaction receipt status: 1
Solution contract balance after: 10 ETH


In [91]:
balance_before = w3.eth.get_balance(SOLVER_ADDRESS)
print(f"Balance before: {Web3.from_wei(balance_before, 'ether')} ETH")

def submit_solution():
    balance = w3.eth.get_balance(SOLUTION_ADDRESS)
    print(f"Addition solution contract balance: {Web3.from_wei(balance, 'ether')} ETH")

    ABI = [
        {"type":"function","name":"submitSolution","inputs":[],"outputs":[],"stateMutability":"nonpayable"}
    ]
    solution_contract = w3.eth.contract(address=SOLUTION_ADDRESS, abi=ABI)
    tx = solution_contract.functions.submitSolution().build_transaction({
        'from': SOLVER_ADDRESS,
        'nonce': w3.eth.get_transaction_count(SOLVER_ADDRESS),
    })
    try:
        gas_estimate = w3.eth.estimate_gas({
            'from': tx['from'],
            'to': tx['to'],
            'data': tx['data'],
        })
        tx['gas'] = gas_estimate
    except Exception as e:
        print(f"Gas estimation failed: {e}")
    signed_tx = w3.eth.account.sign_transaction(tx, private_key=SOLVER_PRIVATE_KEY)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    print(f"Solution submitted! Transaction hash: {tx_hash.hex()}")
    receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    if receipt.status == 1:
        print("Solution submission was successful!")

        balance_after = w3.eth.get_balance(SOLUTION_ADDRESS)
        print(f"Addition solution contract balance after submission: {Web3.from_wei(balance_after, 'ether')} ETH")
    else:
        print("Solution submission failed!")

def withdraw_rewards():
    ABI = [
        {"type":"function","name":"withdraw","inputs":[],"outputs":[],"stateMutability":"nonpayable"}
    ]
    solution_contract = w3.eth.contract(address=SOLUTION_ADDRESS, abi=ABI)
    tx = solution_contract.functions.withdraw().build_transaction({
        'from': SOLVER_ADDRESS,
        'nonce': w3.eth.get_transaction_count(SOLVER_ADDRESS),
    })
    try:
        gas_estimate = w3.eth.estimate_gas({
            'from': tx['from'],
            'to': tx['to'],
            'data': tx['data'],
        })
        tx['gas'] = gas_estimate
    except Exception as e:
        print(f"Gas estimation failed: {e}")
    signed_tx = w3.eth.account.sign_transaction(tx, private_key=SOLVER_PRIVATE_KEY)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    print(f"Withdraw called! Transaction hash: {tx_hash.hex()}")
    receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    if receipt.status == 1:
        print("Withdraw was successful!")
    else:
        print("Withdraw failed!")

submit_solution()
withdraw_rewards()

Balance before: 9999999999999999998.999286691097397682 ETH
Addition solution contract balance: 10 ETH
Solution submitted! Transaction hash: fbf50560f732b85ae3de853ecfc431217fe2adda0304f6bc2469f7abeb16febc
Solution submission was successful!
Addition solution contract balance after submission: 100000010 ETH
Withdraw called! Transaction hash: 817ef64767b7f21a437db30394f59e8825ebabdde24e1098d1b3f8b2c7ab14c4
Withdraw was successful!


In [92]:
balance_after = w3.eth.get_balance(SOLVER_ADDRESS)
print(f"Solver balance: {Web3.from_wei(balance_after, 'ether')} ETH")

if balance_after > balance_before:
    print("Solver's balance has increased.")

Solver balance: 10000000000100000008.999084442094000302 ETH
Solver's balance has increased.


In [93]:
problem_id = PROBLEM_ID
problem_exists = contract.functions.problemExists(problem_id).call()
if problem_exists:
    print(f"Problem {problem_id} exists.")
    prize = contract.functions.getProblemPrize(problem_id).call()
    print(f"Prize for problem {problem_id}: {Web3.from_wei(prize, 'ether')} ETH")
    is_solved = contract.functions.isProblemSolved(problem_id).call()
    print(f"Is problem {problem_id} solved? {'Yes' if is_solved else 'No'}")

Problem 1 exists.
Prize for problem 1: 100000000 ETH
Is problem 1 solved? Yes
